In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# ----------------------------
# 1. Load data
# ----------------------------
df = pd.read_csv("spam.csv", encoding="latin-1")
df = df[["v1", "v2"]]
df = df.rename(columns={"v1": "label", "v2": "text"})

le = LabelEncoder()
df["label"] = le.fit_transform(df["label"])

X_train, X_test, y_train, y_test = train_test_split(
    df["text"],
    df["label"],
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

# ----------------------------
# 2. Tokenization
# ----------------------------
VOCAB_SIZE = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_pad = pad_sequences(
    tokenizer.texts_to_sequences(X_train),
    maxlen=MAX_LEN,
    padding="post"
)

X_test_pad = pad_sequences(
    tokenizer.texts_to_sequences(X_test),
    maxlen=MAX_LEN,
    padding="post"
)

# ----------------------------
# 3. CNN model builder
# ----------------------------
def build_cnn_model(
    embed_dim=128,
    filters=128,
    kernel_size=5,
    dropout_rate=0.5,
    learning_rate=1e-3
):
    model = Sequential([
        Embedding(VOCAB_SIZE, embed_dim),
        Conv1D(filters=filters, kernel_size=kernel_size, activation="relu"),
        GlobalMaxPooling1D(),
        Dropout(dropout_rate),
        Dense(64, activation="relu"),
        Dense(1, activation="sigmoid")
    ])

    model.compile(
        loss="binary_crossentropy",
        optimizer=Adam(learning_rate=learning_rate),
        metrics=["accuracy"]
    )

    return model

# ----------------------------
# 4. Hyperparameter grid
# ----------------------------
param_grid = {
    "embed_dim": [64, 128],
    "filters": [64, 128],
    "kernel_size": [3, 5],
    "dropout_rate": [0.3, 0.5],
    "learning_rate": [1e-3, 3e-4],
    "batch_size": [32, 64]
}

# ----------------------------
# 5. Grid search
# ----------------------------
best_f1 = 0
best_params = None

for embed_dim in param_grid["embed_dim"]:
    for filters in param_grid["filters"]:
        for kernel_size in param_grid["kernel_size"]:
            for dropout_rate in param_grid["dropout_rate"]:
                for lr in param_grid["learning_rate"]:
                    for batch_size in param_grid["batch_size"]:

                        print(
                            f"Training CNN | embed={embed_dim}, filters={filters}, "
                            f"kernel={kernel_size}, dropout={dropout_rate}, "
                            f"lr={lr}, batch={batch_size}"
                        )

                        model = build_cnn_model(
                            embed_dim=embed_dim,
                            filters=filters,
                            kernel_size=kernel_size,
                            dropout_rate=dropout_rate,
                            learning_rate=lr
                        )

                        model.fit(
                            X_train_pad,
                            y_train,
                            epochs=5,
                            batch_size=batch_size,
                            validation_split=0.1,
                            verbose=0
                        )

                        y_pred = (model.predict(X_test_pad) > 0.5).astype(int)
                        f1 = f1_score(y_test, y_pred)

                        print(f"F1 Score: {f1:.4f}\n")

                        if f1 > best_f1:
                            best_f1 = f1
                            best_params = {
                                "embed_dim": embed_dim,
                                "filters": filters,
                                "kernel_size": kernel_size,
                                "dropout_rate": dropout_rate,
                                "learning_rate": lr,
                                "batch_size": batch_size
                            }

# ----------------------------
# 6. Best result
# ----------------------------
print("🔥 Best CNN Model")
print(f"Best F1 Score: {best_f1:.4f}")
print("Best Hyperparameters:")
for k, v in best_params.items():
    print(f"{k}: {v}")


Training CNN | embed=64, filters=64, kernel=3, dropout=0.3, lr=0.001, batch=32
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
F1 Score: 0.9589

Training CNN | embed=64, filters=64, kernel=3, dropout=0.3, lr=0.001, batch=64
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
F1 Score: 0.9488

Training CNN | embed=64, filters=64, kernel=3, dropout=0.3, lr=0.0003, batch=32
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
F1 Score: 0.9225

Training CNN | embed=64, filters=64, kernel=3, dropout=0.3, lr=0.0003, batch=64
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
F1 Score: 0.9078

Training CNN | embed=64, filters=64, kernel=3, dropout=0.5, lr=0.001, batch=32
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
F1 Score: 0.9589

Training CNN | embed=64, filters=64, kernel=3, dropout=0.5, lr=0.001, batch=64
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
F1 Score: 0.9586

Training CNN | embed=64, filters=64, kernel=3, dropout=0.5, lr=0.0003, batch=32
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
F1 Score: 0.9306

Training CNN | embed=64, filters=64, k

In [4]:
# ----------------------------
# 6. Evaluate the model
# ----------------------------
y_pred_prob = model.predict(X_test_pad)
th= [ 40, 45, 50, 55, 60, 65, 70, 75, 80, 85,90]
for th in th:
    y_pred = (y_pred_prob > th/100).astype(int).ravel()
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"Threshold: {th}%")
    print(f"Accuracy : {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall   : {recall:.4f}")
    print(f"F1-score : {f1:.4f}")

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Threshold: 40%
Accuracy : 0.9749
Precision: 0.9060
Recall   : 0.9060
F1-score : 0.9060
Threshold: 45%
Accuracy : 0.9749
Precision: 0.9060
Recall   : 0.9060
F1-score : 0.9060
Threshold: 50%
Accuracy : 0.9767
Precision: 0.9184
Recall   : 0.9060
F1-score : 0.9122
Threshold: 55%
Accuracy : 0.9758
Precision: 0.9178
Recall   : 0.8993
F1-score : 0.9085
Threshold: 60%
Accuracy : 0.9767
Precision: 0.9301
Recall   : 0.8926
F1-score : 0.9110
Threshold: 65%
Accuracy : 0.9767
Precision: 0.9301
Recall   : 0.8926
F1-score : 0.9110
Threshold: 70%
Accuracy : 0.9767
Precision: 0.9301
Recall   : 0.8926
F1-score : 0.9110
Threshold: 75%
Accuracy : 0.9794
Precision: 0.9500
Recall   : 0.8926
F1-score : 0.9204
Threshold: 80%
Accuracy : 0.9803
Precision: 0.9568
Recall   : 0.8926
F1-score : 0.9236
Threshold: 85%
Accuracy : 0.9812
Precision: 0.9638
Recall   : 0.8926
F1-score : 0.9268
Threshold: 90%
Accuracy : 0.9803
Precision: 0.9635
Recall   : 0.8859
F1-score : 0.9231


In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# ----------------------------
# 1. Load data
# ----------------------------
df = pd.read_csv("spam.csv", encoding="latin-1")
df = df[["v1", "v2"]]
df = df.rename(columns={"v1": "label", "v2": "text"})

le = LabelEncoder()
df["label"] = le.fit_transform(df["label"])

X_train, X_test, y_train, y_test = train_test_split(
    df["text"],
    df["label"],
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

# ----------------------------
# 2. Tokenization
# ----------------------------
VOCAB_SIZE = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_pad = pad_sequences(
    tokenizer.texts_to_sequences(X_train),
    maxlen=MAX_LEN,
    padding="post"
)

X_test_pad = pad_sequences(
    tokenizer.texts_to_sequences(X_test),
    maxlen=MAX_LEN,
    padding="post"
)

# ----------------------------
# 3. CNN model builder
# ----------------------------
def build_cnn_model(
    embed_dim=128,
    filters=128,
    kernel_size=5,
    dropout_rate=0.5,
    learning_rate=1e-3
):
    model = Sequential([
        Embedding(VOCAB_SIZE, embed_dim),
        Conv1D(filters=filters, kernel_size=kernel_size, activation="relu"),
        GlobalMaxPooling1D(),
        Dropout(dropout_rate),
        Dense(64, activation="relu"),
        Dense(1, activation="sigmoid")
    ])

    model.compile(
        loss="binary_crossentropy",
        optimizer=Adam(learning_rate=learning_rate),
        metrics=["accuracy"]
    )

    return model
# ----------------------------
# 4. Build model with best hyperparameters
# ----------------------------
best_params = {
    "embed_dim": 128,
    "filters": 128,
    "kernel_size": 3,
    "dropout_rate": 0.5,
    "learning_rate": 0.001
}

model = build_cnn_model(
    embed_dim=best_params["embed_dim"],
    filters=best_params["filters"],
    kernel_size=best_params["kernel_size"],
    dropout_rate=best_params["dropout_rate"],
    learning_rate=best_params["learning_rate"]
)

model.summary()

# ----------------------------
# 5. Train the model
# ----------------------------
history = model.fit(
    X_train_pad,
    y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.1,
    verbose=1
)

# ----------------------------
# 6. Evaluate the model
# ----------------------------
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.7).astype(int).ravel()
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")



Model: "sequential_66"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_66 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_66 (Conv1D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_66         │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_66 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_132 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_133 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - accuracy: 0.8611 - loss: 0.4010 - val_accuracy: 0.8453 - val_loss: 0.3073
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - accuracy: 0.9459 - loss: 0.1328 - val_accuracy: 0.9843 - val_loss: 0.0712
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - accuracy: 0.9920 - loss: 0.0282 - val_accuracy: 0.9843 - val_loss: 0.0526
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 0.9988 - loss: 0.0075 - val_accuracy: 0.9865 - val_loss: 0.0446
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.9990 - loss: 0.0037 - val_accuracy: 0.9865 - val_loss: 0.0456
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.9995 - loss: 0.0024 - val_accuracy: 0.9843 - val_loss: 0.0464
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 0.9888 - val_loss: 0.0477
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 1.0000 - loss: 5.0068e-04 - val_accuracy: 0.9910

In [ ]:
class_counts = df['label'].value_counts()
print("Class distribution:")
print(class_counts)

class_percent = df['label'].value_counts(normalize=True) * 100
print("\nClass distribution in percentage:")
print(class_percent)

Class distribution:
label
0    4825
1     747
Name: count, dtype: int64

Class distribution in percentage:
label
0    86.593683
1    13.406317
Name: proportion, dtype: float64
